In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 
                'occupation', 'relationship', 'race','sex','capital_gain','capital_loss',
                'hours_per_week','native_country','income']

adult_train_df = pd.read_csv('./uci_repos/adults/adult.data',names=column_names, na_values=['?'])
adult_test_df = pd.read_csv('./uci_repos/adults/adult.test', names=column_names,skiprows=[0], na_values=['?']) 

In [3]:
len(adult_train_df), len(adult_test_df)

(32561, 16281)

# Preprocessing

In [4]:
adult_test_df = adult_test_df.drop(['education','age'], axis=1)
adult_train_df = adult_train_df.drop(['education', 'age'], axis=1)

adult_test_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,Private,226802,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,Private,89814,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,Local-gov,336951,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,Private,160323,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,?,103497,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,Private,215419,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,?,321403,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,Private,374983,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,Private,83891,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [5]:
scaler = StandardScaler()
scale_columns = ['fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']
adult_test_df[scale_columns] = scaler.fit_transform(adult_test_df[scale_columns])
adult_train_df[scale_columns] = scaler.fit_transform(adult_train_df[scale_columns])

for feature in ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']:
    le = LabelEncoder()
    adult_train_df[feature] = le.fit_transform(adult_train_df[feature])
    adult_test_df[feature] = le.fit_transform(adult_test_df[feature])

In [6]:
adult_train_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,7,-1.063611,1.134739,4,1,1,4,1,0.148453,-0.21666,-0.035429,39,0
1,6,-1.008707,1.134739,2,4,0,4,1,-0.145920,-0.21666,-2.222153,39,0
2,4,0.245079,-0.420060,0,6,1,4,1,-0.145920,-0.21666,-0.035429,39,0
3,4,0.425801,-1.197459,2,6,0,2,1,-0.145920,-0.21666,-0.035429,39,0
4,4,1.408176,1.134739,2,10,5,2,0,-0.145920,-0.21666,-0.035429,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,4,0.639741,0.746039,2,13,5,4,0,-0.145920,-0.21666,-0.197409,39,0
32557,4,-0.335433,-0.420060,2,7,0,4,1,-0.145920,-0.21666,-0.035429,39,1
32558,4,-0.358777,-0.420060,6,1,4,4,0,-0.145920,-0.21666,-0.035429,39,0
32559,4,0.110960,-0.420060,4,1,3,4,1,-0.145920,-0.21666,-1.655225,39,0


In [7]:
adult_test_df

,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,4,0.353474,-1.196864,4,7,3,2,1,-0.142662,-0.218062,-0.031432,38,0
1,4,-0.942391,-0.417886,2,5,0,4,1,-0.142662,-0.218062,0.769918,38,0
2,2,1.395450,0.750582,2,11,0,4,1,-0.142662,-0.218062,-0.031432,38,1
3,4,-0.275397,-0.028397,2,7,0,2,1,0.871091,-0.218062,-0.031432,38,1
4,0,-0.812954,-0.028397,4,0,3,4,0,-0.142662,-0.218062,-0.832781,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,4,0.245794,1.140071,0,10,1,4,0,-0.142662,-0.218062,-0.351972,38,0
16277,0,1.248371,-0.417886,6,0,2,2,1,-0.142662,-0.218062,-0.031432,38,0
16278,4,1.755221,1.140071,2,10,0,4,1,-0.142662,-0.218062,0.769918,38,0
16279,4,-0.998420,1.140071,0,1,3,1,1,0.576644,-0.218062,-0.031432,38,0


In [8]:
X_train = adult_train_df.drop(columns=['income']).to_numpy()
y_train = (adult_train_df.income).to_numpy()
X_test= adult_test_df.drop(columns=['income']).to_numpy()
y_test = (adult_test_df.income).to_numpy()

In [9]:
'train data :', X_train.shape, 'test data :', X_test.shape

('train data :', (32561, 12), 'test data :', (16281, 12))

In [10]:
y_test

array([0, 0, 1, ..., 0, 0, 1])

In [11]:
len(adult_test_df.native_country.unique())

41

In [12]:
def group_all_labels(data, num=100, minor=[]):
    # this function is to limit the number of labels that are used
    # it returns the indexes according the labels
    # data is an array of labels
    '''

    :param data: array of labels
    :param num: number required
    :param minor: list of minority indexes
    :return: array of labels indexes
    '''

    labels = np.unique(data)
    co_l = []
    min_col =[]
    if not minor:
        for l in labels:
            el_l = np.where(np.array(data) == l)
            co_l.append(el_l[0])

    else:
        for l in labels:
            if l in minor:
                el_l = np.where(np.array(data) == l)
                #co_l.append((el_l[0])[:num])
                #min_col.append((el_l[0])[:num])
                
                ind = np.random.choice(el_l[0], num)
                co_l.append(ind)
                min_col.append(ind)
    
            else:
                el_l = np.where(np.array(data) == l)
                co_l.append(el_l[0])
    return co_l, min_col

In [13]:
grouped_labels, min_label = group_all_labels(y_train, 3000, [1])
gr_data = []
gr_labels = [] 
for index, q in enumerate(grouped_labels):
    print('class {} : number of samples : {}'.format(index,len(q)))
    for r in q:
        gr_data.append(X_train[r])
        gr_labels.append(y_train[r])


class 0 : number of samples : 24720
class 1 : number of samples : 3000


In [14]:
X_train = np.array(gr_data)
y_train = np.array(gr_labels)

X_train.shape, y_train.shape

((27720, 12), (27720,))

# VAE Model 

In [15]:
import tensorflow.compat.v1  as tf
tf.disable_v2_behavior()
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
import numpy as np

mb_size = 32
z_dim = 3
X_dim = 12
y_dim = 2
h_dim = 3
lr = 1e-3

    
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random.normal(shape=size, stddev=xavier_stddev)

X = tf.keras.Input(shape=(X_dim,))
c = tf.keras.Input(shape=(y_dim,))
z = tf.keras.Input(shape=(z_dim,))

Q_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))


def Q(X, c):
    inputs = tf.concat(axis=1, values=[X, c])
    h = tf.nn.relu(tf.matmul(inputs, Q_W1) + Q_b1)
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random.normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps

P_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
P_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

P_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
P_b2 = tf.Variable(tf.zeros(shape=[X_dim]))


def P(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    h = tf.nn.relu(tf.matmul(inputs, P_W1) + P_b1)
    logits = tf.matmul(h, P_W2) + P_b2
    prob = tf.nn.sigmoid(logits)
    return prob, logits

z_mu, z_logvar = Q(X, c)
z_sample = sample_z(z_mu, z_logvar)
_, logits = P(z_sample, c)

X_samples, _ = P(z, c)

recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
vae_loss = tf.reduce_mean(recon_loss + kl_loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
def generate_sample():
    samples = []
    gen_labels =[]
    for r in range(100):
        for index in range(y_dim):
            gen_labels = gen_labels + [1]*mb_size #1 for >50
            y = np.zeros([mb_size, y_dim])
            y[range(mb_size), 1] = 1 #1 for >50 class only
            samples.extend(sess.run(X_samples,
                                   feed_dict={z: np.random.randn(mb_size, z_dim), c: y}))

    gen_samples = np.array(samples).round(decimals=2)
    gen_labels = np.array(gen_labels)
    print(gen_samples.shape)
    print(gen_labels.shape)
    
    return gen_samples, gen_labels

# MLP model

In [17]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Flatten
from tensorflow.keras.models import Sequential


def build_model(input_shape=(12,), num_classes=2):
    """

    :param input_shape: shape of input_data
    :param num_classes: number of classes
    :return: keras.model.sequential compiled with categorical cross-entropy loss
    """
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation="relu"),
        BatchNormalization(),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Flatten(),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.summary()
    return model

# Baseline Experiment

In [18]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

baseline_list =[]
for i in range(10):

    baseline_model = build_model()
    batch_size=32
    epochs=2
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
    history_baseline = baseline_model.fit(X_train, y_train_oh, batch_size=batch_size, 
                        epochs=epochs, validation_data=(X_test, test_y))
    score_baseline = baseline_model.evaluate(X_test, test_y, verbose=0)
    print('baseline test loss: ',score_baseline[0])
    print('baseline test accuracy: ', score_baseline[1] )

    y_pred_baseline_oh = baseline_model.predict(X_test)
    y_pred_baseline = y_pred_baseline_oh.argmax(axis=-1)
    baseline_list.append(classification_report(y_test, y_pred_baseline, output_dict=True))

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                416       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
__________________________________________________

2021-10-24 13:33:39.089001: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2021-10-24 13:33:39.089058: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: rockbook
2021-10-24 13:33:39.089070: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: rockbook
2021-10-24 13:33:39.089202: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.74.0
2021-10-24 13:33:39.089243: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.63.1
2021-10-24 13:33:39.089254: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.63.1 does not match DSO version 470.74.0 -- cannot find working devices in this configuration
2021-10-24 13:33:39.089703: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow

26560/27720 [===========================>..] - ETA: 0s - loss: 0.3935 - acc: 0.8464

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


27720/27720 [==============================] - 2s 60us/sample - loss: 0.3889 - acc: 0.8484 - val_loss: 0.4321 - val_acc: 0.8071
Epoch 2/2
27720/27720 [==============================] - 1s 53us/sample - loss: 0.2685 - acc: 0.8995 - val_loss: 0.4152 - val_acc: 0.8054
baseline test loss:  0.4151529376040102
baseline test accuracy:  0.80541736
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                416       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
__________________________________________

...

27720/27720 [==============================] - 2s 77us/sample - loss: 0.4034 - acc: 0.8426 - val_loss: 0.4823 - val_acc: 0.7921
Epoch 2/2
27720/27720 [==============================] - 2s 69us/sample - loss: 0.2746 - acc: 0.8986 - val_loss: 0.4329 - val_acc: 0.8027
baseline test loss:  0.4328853580188886
baseline test accuracy:  0.8027148
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_16 (Batc (None, 32)                128       
_________________________________________________________________
dense_25 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_17 (Batc (None, 64)                256       
___________________________________________

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


27720/27720 [==============================] - 2s 80us/sample - loss: 0.4152 - acc: 0.8421 - val_loss: 0.4515 - val_acc: 0.8136
Epoch 2/2
27720/27720 [==============================] - 2s 65us/sample - loss: 0.2774 - acc: 0.8994 - val_loss: 0.5213 - val_acc: 0.7877
baseline test loss:  0.5212851481039806
baseline test accuracy:  0.7876666
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_18 (Batc (None, 32)                128       
_________________________________________________________________
dense_28 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_19 (Batc (None, 64)                256       
___________________________________________

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


27720/27720 [==============================] - 2s 81us/sample - loss: 0.3962 - acc: 0.8398 - val_loss: 0.4796 - val_acc: 0.7906
Epoch 2/2
27720/27720 [==============================] - 2s 67us/sample - loss: 0.2691 - acc: 0.8996 - val_loss: 0.4427 - val_acc: 0.8007
baseline test loss:  0.4426691509638201
baseline test accuracy:  0.8006879


In [19]:
total_df = pd.DataFrame(baseline_list[0]).transpose()
print('number of runs: {}'.format(len(baseline_list)))
for r_dict in baseline_list[1:]:
    temp = pd.DataFrame(r_dict).transpose()
    total_df = total_df.add(temp)
    
average_10x = total_df/10.0
average_10x

number of runs: 10


,precision,recall,f1-score,support
0,0.794203,0.996261,0.883816,12435.00000
1,0.934622,0.165107,0.279370,3846.00000
accuracy,0.799920,0.799920,0.799920,0.79992
macro avg,0.864412,0.580684,0.581593,16281.00000
weighted avg,0.827373,0.799920,0.741030,16281.00000


# Augmentation Experiment

In [20]:
solver = tf.compat.v1.train.AdamOptimizer().minimize(vae_loss)
sess = tf.compat.v1.Session ()
sess.run(
tf.compat.v1.global_variables_initializer())

i = 0

for it in tqdm(range(50000)):
    ind = np.random.choice(X_train.shape[0], mb_size)
    X_mb = np.array(X_train[ind])
    y_mb = np.array(y_train_oh[ind])

    _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb, c: y_mb})

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:41<00:00, 1215.17it/s]


In [21]:
augment_list =[]
for i in range(10):
    y_train_oh = np.array(tf.keras.utils.to_categorical(y_train, num_classes=y_dim, dtype='float32'))
    test_y = np.array(tf.keras.utils.to_categorical(y_test, num_classes=y_dim, dtype='float32'))
        
    gen_samples, gen_labels = generate_sample()
    x = np.concatenate([X_train, gen_samples])
    y = np.concatenate([y_train, gen_labels])
    x = np.array(x)
    y_oh = np.array(tf.keras.utils.to_categorical(y, num_classes=y_dim, dtype='float32'))
    
    aug_model = build_model()
    batch_size=32
    epochs=2
    hist = aug_model.fit(x, y_oh, batch_size=batch_size, epochs=epochs, 
                            validation_data=(X_test, test_y))
    y_pred_aug_oh = aug_model.predict(X_test)
    y_pred_aug = y_pred_aug_oh.argmax(axis=-1)
    augment_list.append(classification_report(y_test, y_pred_aug, output_dict=True) )

(6400, 12)
(6400,)
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_20 (Batc (None, 32)                128       
_________________________________________________________________
dense_31 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_21 (Batc (None, 64)                256       
_________________________________________________________________
flatten_10 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


34120/34120 [==============================] - 3s 84us/sample - loss: 0.3067 - acc: 0.8877 - val_loss: 0.4663 - val_acc: 0.7922
Epoch 2/2
34120/34120 [==============================] - 2s 72us/sample - loss: 0.2288 - acc: 0.9159 - val_loss: 0.4337 - val_acc: 0.8110
(6400, 12)
(6400,)
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_22 (Batc (None, 32)                128       
_________________________________________________________________
dense_34 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_23 (Batc (None, 64)                256       
_________________________________________________________________
flatten_11 (Flatten)         (No

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


34120/34120 [==============================] - 3s 87us/sample - loss: 0.3003 - acc: 0.8921 - val_loss: 0.4198 - val_acc: 0.8219
Epoch 2/2
34120/34120 [==============================] - 3s 79us/sample - loss: 0.2272 - acc: 0.9191 - val_loss: 0.4259 - val_acc: 0.8080
(6400, 12)
(6400,)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_24 (Batc (None, 32)                128       
_________________________________________________________________
dense_37 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_25 (Batc (None, 64)                256       
_________________________________________________________________
flatten_12 (Flatten)         (No

...

34120/34120 [==============================] - 4s 104us/sample - loss: 0.3018 - acc: 0.8947 - val_loss: 0.5377 - val_acc: 0.7868
Epoch 2/2
34120/34120 [==============================] - 3s 79us/sample - loss: 0.2275 - acc: 0.9177 - val_loss: 0.4060 - val_acc: 0.8125
(6400, 12)
(6400,)
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_36 (Batc (None, 32)                128       
_________________________________________________________________
dense_55 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_37 (Batc (None, 64)                256       
_________________________________________________________________
flatten_18 (Flatten)         (N

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


34120/34120 [==============================] - 4s 109us/sample - loss: 0.2920 - acc: 0.8958 - val_loss: 0.4572 - val_acc: 0.7974
Epoch 2/2
34120/34120 [==============================] - 3s 80us/sample - loss: 0.2214 - acc: 0.9176 - val_loss: 0.4984 - val_acc: 0.7956
(6400, 12)
(6400,)
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 32)                416       
_________________________________________________________________
batch_normalization_38 (Batc (None, 32)                128       
_________________________________________________________________
dense_58 (Dense)             (None, 64)                2112      
_________________________________________________________________
batch_normalization_39 (Batc (None, 64)                256       
_________________________________________________________________
flatten_19 (Flatten)         (N

/media/adamu/data/projects/notebooks/UCI_encoders/imbalance_venv/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


34120/34120 [==============================] - 4s 104us/sample - loss: 0.3100 - acc: 0.8910 - val_loss: 0.4862 - val_acc: 0.7867
Epoch 2/2
34120/34120 [==============================] - 3s 82us/sample - loss: 0.2311 - acc: 0.9163 - val_loss: 0.4401 - val_acc: 0.8092


In [22]:
import pandas as pd
total_df = pd.DataFrame(augment_list[0]).transpose()
print(len(augment_list))
for r_dict in augment_list[1:]:
    temp = pd.DataFrame(r_dict).transpose()
    total_df = total_df.add(temp)
    
average_10x = total_df/10.0
average_10x

10


,precision,recall,f1-score,support
0,0.804165,0.990776,0.887704,12435.000000
1,0.891629,0.219059,0.347203,3846.000000
accuracy,0.808476,0.808476,0.808476,0.808476
macro avg,0.847897,0.604917,0.617453,16281.000000
weighted avg,0.824827,0.808476,0.760023,16281.000000
